In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../")

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [ ]:
import pandas as pd
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

In [ ]:
# [pin]
file_path = "../data/data-hard.csv"
root_path = "../data/"


df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)
df.sample(20)

,input_text,target_text,prefix
2095,Акция «Попробуйте инвестиции!» с брокером Тинь...,236-3,clsorg
6778,🛢🇷🇺#GAZP «ГАЗПРОМ» МОЖЕТ НАЧАТЬ ПОСТАВКИ ГАЗА ...,48-4,clsorg
2798,Ожидаемые события на 15 марта 🇷🇺 В России По...,225-3,clsorg
6387,📊 ПАО «НК Роснефть» Тикер: ROSN Идея: Long ⬆️ ...,112-5,clsorg
5512,🌎#алюминий #металлы мировые производители алю...,11-4,clsorg
2225,Взгляд на компанию 📑 В пятницу Норникель отчи...,53-3,clsorg
6772,🛢🇰🇿🇷🇺#TRNFP Казахстан в 2024 году может нарас...,175-4,clsorg
3294,Тинькофф $TCS $TCSG продолжит акцию по компенс...,225-4,clsorg
5192,🇷🇺#SPBE #отчетность,255-3,clsorg
5934,💥🇷🇺#MTLRp = макс за 11 мес Китай увеличил им...,99-4;129-4,clsorg


In [ ]:
m_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

In [ ]:
from src.t5.dataset import NERDataModel
from src.t5.model import NERModel

BATCH_SIZE = 64
EPOCHS = 10
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
data_module = NERDataModel(
    train_df, test_df, tokenizer, batch_size=BATCH_SIZE, source_max_token_length=512
)
data_module.setup()

In [ ]:
model = NERModel()

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="ner",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=EPOCHS,
    accelerator="cuda",
)

In [ ]:
!rm -r lightning_logs

In [ ]:
trainer.fit(model, data_module)

In [ ]:
# trained_model = NERModel.load_from_checkpoint("checkpoints/ner-v5.ckpt")
trained_model = NERModel.load_from_checkpoint("checkpoints/ner-v6.ckpt")
trained_model.freeze()

In [ ]:
import torch

from src.t5.utils import evaluate_metric, generate_answer_batched

with torch.inference_mode(), torch.cuda.amp.autocast():
    predictions = generate_answer_batched(
        trained_model=trained_model.model,
        tokenizer=tokenizer,
        data=test_df[:1200],
        batch_size=64,
        max_length=256,
    )

In [ ]:
ldf = test_df.copy()[:1200]
ldf["predictions"] = predictions
ldf[["tcomp", "tsent"]] = (
    ldf["target_text"].str.split(";", expand=True)[0].str.split("-", expand=True)
)
ldf[["pcomp", "psent"]] = (
    ldf["predictions"].str.split(";", expand=True)[0].str.split("-", expand=True)
)

In [ ]:
# [pin]

evaluate_metric(
    company_predictions=ldf["pcomp"].tolist(),
    company_labels=ldf["tcomp"].tolist(),
    sentiment_predictions=ldf["psent"].tolist(),
    sentiment_labels=ldf["tsent"].tolist(),
)

{'total': 55.85067410545177,
 'f1': 0.618680148775702,
 'accuracy': 0.49833333333333335}